In [8]:
import os

import cv2
from src.models.AENet import Predictor

IMG_SIZE = 224

In [9]:
predictor = Predictor()

/home/pessoal/Documents/repos/liveness_system/src/models/AENet.py:219: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(base_ckpt_path + "ckpt_iter.pth.

In [10]:
base_path = "/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/"
live_paths = [
    os.path.join(base_path, "4930/live/504311.png"),
    os.path.join(base_path, "4930/live/511717.png"),
    os.path.join(base_path, "4931/live/494918.png"),
    os.path.join(base_path, "4943/live/494437.png"),
    os.path.join(base_path, "4973/live/494955.png"),
]
spoof_paths = [
    os.path.join(base_path, "4930/spoof/495600.png"),
    os.path.join(base_path, "3613/spoof/541354.png"),
    os.path.join(base_path, "4931/spoof/497474.png"),
    os.path.join(base_path, "4943/spoof/494824.png"),
    os.path.join(base_path, "4973/spoof/502204.png"),
]
bbox_lives = ["".join(path.split(".")[:-1]) + "_BB.txt" for path in live_paths]
bbox_spoofs = ["".join(path.split(".")[:-1]) + "_BB.txt" for path in spoof_paths]

print(live_paths)
print(spoof_paths)
print(bbox_lives)
print(bbox_spoofs)

['/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/live/504311.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/live/511717.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4931/live/494918.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4943/live/494437.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4973/live/494955.png']
['/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/spoof/495600.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/3613/spoof/541354.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4931/spoof/497474.p

In [11]:
live_images = []
spoof_images = []
bb_lives = []
bb_spoofs = []

for live_path, spoof_path, bb_live, bb_spoof in zip(live_paths, spoof_paths, bbox_lives, bbox_spoofs):
    # Load the images
    live_images.append(cv2.imread(live_path))
    spoof_images.append(cv2.imread(spoof_path))

    # Extract bounding boxes
    with open(bb_live, "r") as file:
        bb_lives.append(list(map(int, file.readlines()[0].split()[:-1])))

    with open(bb_spoof, "r") as file:
        bb_spoofs.append(list(map(int, file.readlines()[0].split()[:-1])))

print(live_images[0].shape)
print(spoof_images[0].shape)
print(bb_lives[0])
print(bb_spoofs[0])
bbox_lives = bb_lives
bbox_spoofs = bb_spoofs

(495, 500, 3)
(450, 600, 3)
[54, 57, 96, 129]
[33, 57, 94, 115]


In [12]:
for i in range(len(live_images)):
    # Resize images
    live_images[i] = cv2.resize(live_images[i], (IMG_SIZE, IMG_SIZE))
    spoof_images[i] = cv2.resize(spoof_images[i], (IMG_SIZE, IMG_SIZE))

    # Crop faces and show cropped images
    bbox_live = bbox_lives[i]
    bbox_spoof = bbox_spoofs[i]

    live_images[i] = live_images[i][
        bbox_live[1] : bbox_live[1] + bbox_live[3],
        bbox_live[0] : bbox_live[0] + bbox_live[2],
    ]
    spoof_images[i] = spoof_images[i][
        bbox_spoof[1] : bbox_spoof[1] + bbox_spoof[3],
        bbox_spoof[0] : bbox_spoof[0] + bbox_spoof[2],
    ]

In [23]:

for live in live_images:
    rst1 = predictor.predict([live])[0]
    print(rst1[1])
for spoof in spoof_images:
    rst2 = predictor.predict([spoof])[0]
    print(rst2[1])

[[[157 164 172]
  [ 78  83 103]
  [ 67  74  97]
  ...
  [ 18  29  51]
  [ 17  29  48]
  [ 26  36  52]]

 [[112 120 135]
  [ 69  77 102]
  [ 74  85 110]
  ...
  [ 21  40  63]
  [ 19  37  55]
  [ 15  30  48]]

 [[101 108 125]
  [ 90  98 117]
  [ 72  81 106]
  ...
  [ 24  41  67]
  [ 14  31  53]
  [ 16  31  51]]

 ...

 [[122 164 213]
  [120 161 212]
  [121 162 214]
  ...
  [219 227 227]
  [207 219 222]
  [200 214 220]]

 [[123 164 213]
  [119 160 211]
  [120 161 213]
  ...
  [214 225 227]
  [204 216 219]
  [203 216 221]]

 [[125 167 216]
  [120 161 212]
  [119 159 212]
  ...
  [209 221 225]
  [202 214 218]
  [206 219 222]]]
0.8958308
[[[ 80 117 153]
  [ 78 116 151]
  [ 78 114 151]
  ...
  [ 71 113 155]
  [ 96 132 174]
  [108 146 187]]

 [[ 69 107 145]
  [ 74 110 149]
  [ 75 112 150]
  ...
  [ 98 138 178]
  [ 96 132 172]
  [113 152 191]]

 [[ 72 108 150]
  [ 82 117 159]
  [ 86 121 161]
  ...
  [118 159 197]
  [ 92 132 172]
  [123 161 200]]

 ...

 [[ 11  29 106]
  [ 14  34 110]
  [ 19  42